<a href="https://colab.research.google.com/github/mian482/Scripts/blob/master/OptionsUnusualActivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

author: u/gigantoir

This notebook screens S&P component stocks for unusual volume relative to open
interest for a given expiration date. Currently, the criteria is open interest 
\> 200 and volume/open interest \> .5. This could be tweaked and I'm open to
feedback but would mostly like if you did that coding yourself. Note that 
historical options data is really hard to find and I can't get change in IV 
although that would be cool. 

Code takes a while to run as it's not super efficient, but you should be good
within like 5 minutes. Download that CSV and do some Shkreli no mouse Excel
speedruns on it.

Feel free to DM me with questions / suggestions but please stop asking me to
teach you how to code. ALSO if you find free money you are required to DM me 
about it.

Have fun autists. Mods r gay. Long live WSB REEEEEEEEEEEEEEEEEEEE


In [0]:
!pip install pandas_datareader
!pip install yfinance

In [0]:
#Imports, keys here are pandas_datareader allows us to download easily and
#yfinance allows us to get into yahoo
import pandas_datareader.data as data
import yfinance as yf
import pandas as pd

yf.pdr_override()

In [0]:
#S&P components list that we'll be looping through. You can add your retarded
#meme stocks here like SPCE if you want
sp_list = ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'EVRG', 'ES', 'RE', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FLIR', 'FLS', 'FMC', 'F', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS', 'GWW', 'HRB', 'HAL', 'HBI', 'HOG', 'HIG', 'HAS', 'HCA', 'PEAK', 'HP', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HFC', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'IR', 'INTC', 'ICE', 'IBM', 'INCY', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'J', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM', 'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KSS', 'KHC', 'KR', 'LB', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LEG', 'LDOS', 'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LYB', 'MTB', 'M', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MKC', 'MXIM', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MHK', 'TAP', 'MDLZ', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'MYL', 'NDAQ', 'NOV', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NBL', 'JWN', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'ORLY', 'OXY', 'ODFL', 'OMC', 'OKE', 'ORCL', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PBCT', 'PEP', 'PKI', 'PRGO', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTN', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SLG', 'SNA', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL', 'FTI', 'TFX', 'TXN', 'TXT', 'TMO', 'TIF', 'TJX', 'TSCO', 'TDG', 'TRV', 'TFC', 'TWTR', 'TSN', 'UDR', 'ULTA', 'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UTX', 'UHS', 'UNM', 'VFC', 'VLO', 'VAR', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VIAC', 'V', 'VNO', 'VMC', 'WRB', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WDC', 'WU', 'WRK', 'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']

In [0]:
import sys, os
from datetime import date


def unusualActivity(calls_or_puts, exp_date, stocklist):
    '''
    unusualActivity scans yahoo finance for a list of stocks and returns contracts showing unusually high volume/open interest

    Args:
    calls_or_puts (str): do you want to return calls or puts?
    exp_date (str): date of contract expiry
    stocklist (list[str]): list of tickers to loop through
    '''
    #We're going to suppress prints b/c datareader is annoying then restore printing so this helps
    old_stdout = sys.stdout
    finaldf = pd.DataFrame()
    for x in stocklist:
        ticker = yf.Ticker(x)
        try:
            opt = ticker.option_chain(exp_date)
            if calls_or_puts == 'calls':
                #Suppress prints
                sys.stdout = open(os.devnull, "w") 
                #Add some info about ticker and UL price to our dataframe
                opt.calls.insert(0, 'Symbol', x)
                opt.calls.insert(3, 'stock_price', data.get_data_yahoo(x, end_date = date.today())['Close'][-1])
                #Calculate our volume/open interest, this is how we define unusual activity
                opt.calls['V/OI'] = (opt.calls['volume'].astype('float')/opt.calls['openInterest']) 
                #Inside of the brackets is where we apply our filter to get the unusual stuff. Feel free to mess with this in your own version
                finaldf = finaldf.append(opt.calls[(opt.calls['volume'].astype('float')/opt.calls['openInterest'] > .5) & (opt.calls['openInterest'] > 200)])
            elif calls_or_puts == 'puts':
                sys.stdout = open(os.devnull, "w") 
                opt.puts.insert(0, 'Symbol', x)
                opt.puts.insert(3, 'stock_price', data.get_data_yahoo(x, end_date = date.today())['Close'][-1])
                opt.puts['stock_price'] = data.get_data_yahoo(x, end_date = date.today())['Close'][-1]
                opt.puts['V/OI'] = (opt.puts['volume'].astype('float')/opt.puts['openInterest'])
                finaldf = finaldf.append(opt.puts[(opt.puts['volume'].astype('float')/opt.puts['openInterest'] > .5) & (opt.puts['openInterest'] > 200)])
            else:
                print('set calls_or_puts equal to calls or puts retard')
                break
        except:
            pass
    sys.stdout = old_stdout
    return finaldf

In [0]:
#Prep our params
calls_or_puts = 'puts'
exp_date = '2020-02-28'

returned = unusualActivity(calls_or_puts, exp_date, sp_list)

#Do some final formatting changes
returned = returned.drop(columns = ['contractSymbol', 'lastTradeDate', 'contractSize', 'currency'])
returned.insert(3, 'Distance OTM', returned['stock_price'] - returned['strike'])
returned['Value'] = returned['openInterest']*returned['lastPrice']*100

returned

,Symbol,stock_price,strike,Distance OTM,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,V/OI,Value
12,MMM,156.929993,152.5,4.429993,0.68,0.59,0.65,0.30,78.9474,127.0,243.0,0.302741,False,0.522634,16524.0
13,MMM,156.929993,155.0,1.929993,1.14,1.16,1.24,0.37,48.0519,594.0,351.0,0.283210,False,1.692308,40014.0
17,ABT,87.449997,88.0,-0.550003,1.20,1.16,1.20,0.80,200,2414.0,296.0,0.219246,True,8.155405,35520.0
25,ADBE,372.950012,355.0,17.950012,1.34,1.27,1.35,0.46,52.2727,425.0,244.0,0.407110,False,1.741803,32696.0
26,ADBE,372.950012,357.5,15.450012,1.59,1.53,1.64,0.44,38.2609,494.0,279.0,0.394293,False,1.770609,44361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,WYNN,127.949997,128.0,-0.050003,3.35,3.20,3.35,2.17,183.898,801.0,242.0,0.543950,True,3.309917,81070.0
24,WYNN,127.949997,130.0,-2.050003,4.25,4.40,4.60,2.20,107.317,1356.0,857.0,0.560795,True,1.582264,364225.0
26,WYNN,127.949997,132.0,-4.050003,6.05,5.85,6.00,3.50,137.255,230.0,400.0,0.583012,True,0.575000,242000.0
9,XLNX,87.720001,83.0,4.720001,0.11,0.25,0.31,0.00,0,266.0,251.0,0.436529,False,1.059761,2761.0


In [0]:
#Download the CSV and you're done!
from google.colab import files

returned.to_csv('unusual_puts_activity_2.28_exp.csv')
files.download('unusual_puts_activity_2.28_exp.csv')